In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('../traces/12queens', header=None)
data.columns = ['branch_id', 'taken']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2727424 entries, 0 to 2727423
Data columns (total 2 columns):
branch_id    int64
taken        int64
dtypes: int64(2)
memory usage: 41.6 MB


## 8192 Entry BHT (13-bit)

In [4]:
class BHT:
    def __init__(self, bits=13):
        self.bits = bits
        self.mask = 2**self.bits - 1
        self.table = np.zeros(2**self.bits, dtype=np.bool)
    
    def key(self, branch_id):
        return branch_id & self.mask
        
    def update(self, branch_id, value):
        self.table[self.key(branch_id)] = value
    
    def get(self, branch_id):
        return self.table[self.key(branch_id)]

In [5]:
x = BHT()
print(x.get(6))
x.update(6, 0)
print(x.get(6))

False
False


In [55]:
bht = BHT()
total_predictions, mispredictions = (data.shape[0],0)

for i in range(data.shape[0]):
    branch_id = data.iloc[i].branch_id
    taken = data.iloc[i].taken
    if bht.get(branch_id) != taken:
        mispredictions = mispredictions + 1
        bht.update(branch_id,taken)
        
print(total_predictions, mispredictions)
print(mispredictions/total_predictions*100)

838802 140128
16.70573031537836


In [6]:
class BHT_2:
    def __init__(self, bits=12):
        self.bits = bits
        self.mask = 2**self.bits - 1
        self.table = np.zeros(2**self.bits, dtype=np.uint8) # only 2 bits are used
        self.states = {(0,0): 0, (0,1): 1,
                       (1,0): 0, (1,1): 3,
                       (2,0): 0, (2,1): 3,
                       (3,0): 2, (3,1): 3}
    
    def key(self, branch_id):
        return branch_id & self.mask
        
    def update(self, branch_id, value):
        self.table[self.key(branch_id)] = self.states[self.table[self.key(branch_id)], value]
    
    def get(self, branch_id):
        return self.table[self.key(branch_id)]//2

In [18]:
bht = BHT_2()
iters = data.shape[0]
total_predictions, mispredictions = (iters,0)#(data.shape[0],0)

for i in range(iters):
    branch_id = data.iloc[i].branch_id
    taken = data.iloc[i].taken
    if bht.get(branch_id) != taken:
        mispredictions = mispredictions + 1
    bht.update(branch_id,taken)
        
# print(total_predictions, mispredictions)
print(mispredictions/total_predictions*100)

34.936


In [14]:
x = BHT_2()
print(x.get(6))
x.update(6, 1)
print(x.get(6))
x.update(6, 1)
print(x.get(6))
x.update(6, 0)
print(x.get(6))

0
0
1
1


In [94]:
(2 << 1 | 1)%4

1

In [84]:
2//2

1

In [67]:
3<<1

6